# RRN-LSTM with Tensorflow

Following the tutorial [A noob’s guide to implementing RNN-LSTM using Tensorflow](http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow).

See [Andrej Karpathy's blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) for more information on RRNs.

# Task
Given a binary string, containing `0`s and `1`s, of length 20, we need to determine the count of `1`s the string.

# Simple Soltuion
This is easy to solve without using a RRN, as shown in the code below. However, this is a simple problem with which to experiment using RRNs.

In [17]:
from functools import reduce

def simple_num_1s(s: str) -> int:
    assert(len(s) == 20)
    
    return reduce((lambda acc, x: acc + x if x == 1 else acc), s)

# Should print out '9'.
print(str(simple_num_1s([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1])))

9


# RRN Solution

- Each input is a binary string of length 10.
- An input will be represented as a python list of `0`s and `1`s.
- There are $2^{10}$ possible combinations of `0`s and `1`s. 
- The network will be trained on all possible combinations.
- The inputs will be shuffled before giving them to the network to train with.

In [50]:
import numpy as np
from random import shuffle
from typing import List

def make_all_binary_strings(length: int = 10) -> List[List[int]]:
    """
    returns: a list of all possible binary strings, where a binary string is represened as a list of ints.
    """
    format_string = '{0:0' + str(length) + 'b}'
    all_binary_strings = [format_string.format(i) for i in range(2**length)]
    shuffle(all_binary_strings)
    return [list(map(int,i)) for i in all_binary_strings]

make_all_binary_strings(length=2)

[[1, 1], [0, 1], [0, 0], [1, 0]]

- Tensorflow requires input as a tensor, i.e. a Tensorflow variable.
- The tensor is 3D with the dimensions [batch_size, sequence_length, input_dimension], where
    - `batch_size` is something we’ll determine later 
    - `sequence_length` is fixed at 10 
    - `input_dimension` is 1 i.e each individual bit of the string. Therefore, each bit will actually be represented as a list containing just that bit. 

In [51]:
def make_training(training_binary_strings: List[List[int]]) -> List[np.array]:
    # Turn every every element into an array containing that one element.
    # This is required as the input_dimension is 1.
    xs = list(map(lambda bits: list(map(lambda bit: [bit], bits)), training_binary_strings))
    # Make each array of bits a numpy array.
    return list(map(lambda bits: np.array(bits), xs))

make_training(make_all_binary_strings(length=2))

[array([[0],
        [0]]), array([[1],
        [0]]), array([[1],
        [1]]), array([[0],
        [1]])]